# Imports


In [1]:
import numpy as np
from math import *
import os
from sklearn.cluster import AffinityPropagation
import random as rd
import pandas as pd

# Clases

In [2]:
class Item:
    def __init__(self,elementos,id):
        self.elementos = elementos
        self.beneficio = 0
        self.peso = 0
        self.unos = 0
        self.q = np.sum(self.elementos)
        self.id = id
        self.score = 0

    def actualizar(self,beneficio,peso):
        self.peso = np.sum(self.elementos*peso)
        self.beneficio = beneficio
        self.unos = np.sum(self.elementos)
        self.q = np.sum(self.elementos)
    
    def repack(self,elementos_mochila):
        self.elementos = self.elementos-elementos_mochila
        self.elementos = np.clip(self.elementos,0,1)

        
    def add(self,vector,beneficio,peso):
        self.elementos = self.elementos+vector
        self.elementos = np.clip(self.elementos,0,1)
        self.beneficio += beneficio
        self.peso = np.sum(self.elementos*peso)
    
    def clonar(self):
        item = Item(self.elementos,self.id)
        item.beneficio = self.beneficio
        item.peso = self.peso
        item.unos = self.unos
        item.q = self.q
        item.score = self.score
        return item
        
    def setScore(self,score):
        self.score = score
        if self.score == 0:
            self.score = 1

class Mochila:
    def __init__(self,capacidad,l):
        self.capacidad = capacidad
        self.ocupado = 0
        self.elementos = np.zeros(l).astype(int)
        self.beneficio = 0
    def add(self,item,peso):
        #Suma self.elementos con vector
        self.elementos = self.elementos + item.elementos
        self.elementos = np.clip(self.elementos, 0, 1)
        self.actualizar(peso)
        self.beneficio += item.beneficio
    def actualizar(self,peso):
        self.ocupado = np.sum(self.elementos*peso)
    def faltante(self):
        return 1-self.elementos
    

# Funciones

In [3]:
def leer(archivo):    
    f = open (archivo,'r')
    flag = 0
    matriz = []
    while True:
        linea = f.readline()
        if not linea:
            break
        linea = linea.split(" ")
        #ITEMS; ELEMENTOS; CAPACIDAD
        if flag == 0:
            items = linea[0]
            elementos = linea[1]
            capacidad = linea[2]
        #BENEFICIO
        if flag == 1:
            linea = linea[:-1]
            beneficio = linea
        if flag == 2:
            linea = linea[:-1]
            peso = linea

        if flag > 2:
            linea = linea[:-1]
            matriz.append(linea)

        flag += 1
    return int(capacidad),np.array(beneficio).astype(int),np.array(peso).astype(int),np.array(matriz).astype(int)

def menu():
    base = 'C:/Users/Levy_Owls/Desktop/Trabajo/SUKP/Three kinds of SUKP instances (EAs)/'
    carpetas = os.listdir(base)
    print("Por favor seleccionar carpeta")
    for i in range(len(carpetas)):
        #Si no hay cosas raras
        if "txt" not in carpetas[i]:
            print(str(i)+")",carpetas[i])         
    sel = int(input(""))
    while sel>3 or sel<1:
        sel = int(input("Lo lamento, asumi que podria con una simple indicacion, intente nuevamente: "))   
    base += carpetas[i]+"/"
    print("\n~°~°~°~Soy un separador~°~°~°~°~°\n")
    archivos = os.listdir(base)
    print("Por favor seleccionar instancia")
    for i in range(len(archivos)):
        print(str(i+1)+")",archivos[i])
    sel = int(input(""))
    while sel>10 or sel<1:
        sel = int(input("Lo lamento, asumi que podria con una simple indicacion, intente nuevamente: "))  
    base += archivos[sel-1]
    #RETORNA LA INSTANCIA
    return base

def score(item,vectores):
    a = 0
    for i in vectores:
        aux = np.where((i.elementos+item.elementos)==2)
        a += len(aux[0])
        
    aux = np.where(item.elementos==1)
    a -= len(aux[0])
    return a*item.beneficio/item.peso


def pp(lista1,lista2):
    total = 0
    for i in range(len(lista1)):
        total += lista1[i]*lista2[i]

    return total

def modulo(l): 
    res = 0 
    for i in range(len(l)):
        res += l[i]**2     
    return sqrt(res)
    
    
def parentesco(l1,l2):
    producto_punto = pp(l1,l2)
    m1 = modulo(l1)
    m2 = modulo(l2)
    
    if m1*m2 != 0:
        angulo = producto_punto/(m1*m2)
    else:
        angulo = producto_punto/-1
    return angulo


#Para su funcionamiento mas veloz deberia crear la mitad superior o inferior y luego duplicar solamente
def matrizDistancia(vectores):
    
    #Crea una matriz de zeros
    ts =  np.zeros((len(vectores),len(vectores)))
    h = 0
    #Genera una matriz triangular superior
    for i in range(len(ts)):
        for j in range(len(ts[0])):
            if j > h:
                ts[h][j]=1-(parentesco(vectores[h].elementos,vectores[j].elementos))
                #ts[h][j] = 2
        h += 1
        
    #Clona lo de la triangular superior, 
    ts = ts + ts.T - np.diag(np.diag(ts))
    return ts

def combinatoria(c, n):
    return [s for s in potencia(c) if len(s) == n]
def potencia(c):
    if len(c) == 0:
        return [[]]
    r = potencia(c[:-1])
    return r + [s + [c[-1]] for s in r]

# Lectura de archivo y obtencion de items

In [4]:
#Se elige el archivo
archivo = menu()
#Se lee la info del archivo
capacidad,beneficio,peso,matriz = leer(archivo)

#Set union KP problem


mochila = Mochila(capacidad,len(matriz[0]))
vectores = []
#Se genera un vector de items  con sus respectivos elementos
for i in range(len(matriz)):
    
    item = Item(np.array(matriz[i]),i)
    item.actualizar(beneficio[i],peso)
    item.repack(mochila.elementos)
    
    vectores.append(item)

Por favor seleccionar carpeta
1) SUKP Instances (m equal n)
2) SUKP Instances (m greater than n)
3) SUKP Instances (m less than n)
1

~°~°~°~Soy un separador~°~°~°~°~°

Por favor seleccionar instancia
1) sukp 185_200_0.10_0.75.txt
2) sukp 185_200_0.15_0.85.txt
3) sukp 285_300_0.10_0.75.txt
4) sukp 285_300_0.15_0.85.txt
5) sukp 385_400_0.10_0.75.txt
6) sukp 385_400_0.15_0.85.txt
7) sukp 485_500_0.10_0.75.txt
8) sukp 485_500_0.15_0.85.txt
9) sukp 85_100_0.10_0.75.txt
10) sukp 85_100_0.15_0.85.txt
1


# Se crea la matriz distancia y se imprime

In [5]:
md = matrizDistancia(vectores)
pmd = pd.DataFrame(md)
pmd

,0,1,2,3,4,5,6,7,8,9,...,175,176,177,178,179,180,181,182,183,184
0,0.000000,0.945767,0.948701,1.000000,0.904654,0.951205,0.920943,0.954356,0.846103,1.000000,...,0.841886,1.000000,0.948701,0.900000,1.000000,0.756025,0.944098,0.950000,1.000000,0.915485
1,0.945767,0.000000,0.944359,0.952435,0.896582,0.841223,0.914251,0.900985,0.888717,0.882353,...,0.942834,0.947074,0.888717,1.000000,0.944359,1.000000,0.939366,1.000000,0.888717,0.770825
2,0.948701,0.944359,0.000000,0.910016,0.902177,0.899875,0.918889,0.906341,1.000000,0.944359,...,0.891852,0.849812,0.894737,0.846103,0.947368,1.000000,1.000000,0.948701,0.947368,0.869934
3,1.000000,0.952435,0.910016,0.000000,1.000000,0.871612,0.861325,0.799840,0.820031,0.714609,...,0.953775,0.828816,0.820031,0.780735,0.730047,0.914408,0.852913,0.868441,0.820031,0.888813
4,0.904654,0.896582,0.902177,1.000000,0.000000,0.906952,0.924622,0.956481,0.902177,0.948291,...,0.949748,0.860427,0.902177,0.856981,1.000000,0.813903,0.893400,0.904654,0.853265,0.838835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.756025,1.000000,1.000000,0.914408,0.813903,1.000000,0.922848,0.821826,1.000000,1.000000,...,0.845697,0.904762,0.899875,0.902410,0.899875,0.000000,0.890891,0.902410,0.899875,0.958761
181,0.944098,0.939366,1.000000,0.852913,0.893400,0.890891,0.823223,0.846907,0.942646,1.000000,...,0.941074,0.945446,0.942646,0.776393,0.942646,0.890891,0.000000,0.944098,0.770584,0.905509
182,0.950000,1.000000,0.948701,0.868441,0.904654,0.902410,0.841886,0.863069,0.948701,0.891535,...,1.000000,0.853615,0.948701,0.850000,0.846103,0.902410,0.944098,0.000000,1.000000,0.830969
183,1.000000,0.888717,0.947368,0.820031,0.853265,0.949937,0.918889,0.953171,1.000000,0.944359,...,0.891852,0.949937,0.947368,0.897402,0.894737,0.899875,0.770584,1.000000,0.000000,0.869934


# Se clusteriza

In [6]:
clustering = AffinityPropagation(random_state=5).fit(md)
c = clustering.labels_
c

array([15, 28,  3, 24, 17,  8,  2,  4, 29,  0,  5, 16,  1,  3, 18, 21, 15,
       24, 22,  9,  2,  5, 12, 26, 16, 20,  2, 15, 13, 23, 19, 22, 20, 25,
        3, 15,  1,  0, 28,  0, 13, 10, 15,  4,  5, 27,  6, 15,  9, 16,  1,
        0,  3, 12,  8, 27, 15, 15,  7, 25, 10, 27, 18, 12,  2, 21, 22,  2,
       16, 23, 17, 22, 23,  6, 12,  8, 19, 26,  9, 11, 17, 27, 16, 14, 11,
       29, 22,  9, 10, 11,  9, 18,  5,  6, 12, 12, 13, 20, 13, 17, 29, 27,
       11, 15, 14, 15, 16, 12,  2,  7,  8, 19, 25, 10,  8,  9, 25,  0, 13,
        2, 17, 18,  4, 27, 20,  8, 15, 26, 21, 27, 20,  5, 19, 19, 20,  1,
       10, 27, 11, 21, 22, 19, 28, 23,  9, 29,  8, 17, 11, 14,  1,  8,  7,
       24,  4, 28, 14, 10, 16,  5,  9, 25,  6, 24,  8, 14, 26, 11, 27, 26,
       28, 24,  4, 26,  4,  5, 11, 29,  7,  0, 23,  1,  9, 13, 13],
      dtype=int64)

# Valores 

In [7]:
np.unique(c)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int64)

# Se crea el diccionario que almacena vectores de igual valor


In [8]:
dic = {}

for i in range(len(vectores)):
    item = vectores[i]
    cluster = c[i]
    #Si no esta lleno de puros 0
    if np.sum(item.elementos) != 0:
        #Si el cluster existe en el dic
        if cluster in dic:
            dic[cluster].append(item)
        else:
            dic[cluster] = [item]


# Para cada cluster se obtiene la data que solicito el profe

In [9]:
resumen = []

for k in dic:
    lista = dic[k]
    for item in lista:
        p = []
        p.append(k)
        p.append(item.id)
        p.append(item.beneficio)
        p.append(item.peso)
        p.append(item.beneficio/item.peso)
        resumen.append(p)
resumen = np.array(resumen)

# Viendo la totalidad por cada clave

In [10]:
pResumen = pd.DataFrame(resumen,columns = ["cluster","id vector","beneficio","costo","beneficio/costo"])
#Se ordena por el cluster
pResumen = pResumen.sort_values(by=['cluster'])


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pResumen.to_string(index=False))

 cluster  id vector  beneficio   costo  beneficio/costo
     0.0       37.0      103.0  4088.0         0.025196
     0.0      179.0      151.0  2612.0         0.057810
     0.0      117.0      216.0  3428.0         0.063011
     0.0        9.0      296.0  2737.0         0.108148
     0.0       39.0      226.0  2488.0         0.090836
     0.0       51.0      327.0  3352.0         0.097554
     1.0       12.0      189.0  2540.0         0.074409
     1.0       36.0      240.0  4298.0         0.055840
     1.0      181.0      218.0  2957.0         0.073723
     1.0      150.0      221.0  3375.0         0.065481
     1.0      135.0      266.0  4610.0         0.057701
     1.0       50.0      417.0  2688.0         0.155134
     2.0       20.0      356.0  3830.0         0.092950
     2.0       26.0      474.0  3698.0         0.128177
     2.0       64.0      247.0  3968.0         0.062248
     2.0       67.0      389.0  3258.0         0.119398
     2.0      108.0      265.0  3702.0         0

# Se revisan en forma de grupo

In [11]:
resumen = []


for k in dic:
    lista = dic[k]
    #Crea un item nuevo y vacio
    item = Item(np.zeros(len(lista[0].elementos)),k)
    item.actualizar(0,0)   
    for i in lista:
        item.add(i.elementos,beneficio[i.id],peso)
        
    p = []
    p.append(k)
    p.append(item.id)
    p.append(item.beneficio)
    p.append(item.peso)
    p.append(item.beneficio/item.peso)
    resumen.append(p)
resumen = np.array(resumen)

pResumen = pd.DataFrame(resumen,columns = ["cluster","id vector","beneficio","costo","beneficio/costo"])
#Se ordena por el cluster
pResumen = pResumen.sort_values(by=['cluster'])


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pResumen.to_string(index=False))

 cluster  id vector  beneficio    costo  beneficio/costo
     0.0        0.0     1319.0  12909.0         0.102177
     1.0        1.0     1551.0  13931.0         0.111334
     2.0        2.0     2592.0  15126.0         0.171361
     3.0        3.0      855.0   8505.0         0.100529
     4.0        4.0      870.0  13284.0         0.065492
     5.0        5.0     1554.0  18430.0         0.084319
     6.0        6.0      973.0   7918.0         0.122885
     7.0        7.0     1314.0  11640.0         0.112887
     8.0        8.0     2260.0  19278.0         0.117232
     9.0        9.0     2235.0  16206.0         0.137912
    10.0       10.0     2128.0  13357.0         0.159317
    11.0       11.0     2322.0  18127.0         0.128096
    12.0       12.0     2354.0  15927.0         0.147799
    13.0       13.0     1904.0  17468.0         0.108999
    14.0       14.0     1261.0  10577.0         0.119221
    15.0       15.0     1848.0  23359.0         0.079113
    16.0       16.0     1561.0 

# Lo de arriba pero creando combinatorias

In [15]:
d2 = {}

for k in dic:
    lista = dic[k]
    combinaciones = []
    for largo in range(len(lista)):
        combinaciones += combinatoria(lista,largo)
    #Una vez se tienen las combinaciones se crean los items
    d2[k] = []
    
    for comb in combinaciones:
        item = Item(np.zeros(len(lista[0].elementos)),k)
        for i in comb:
            item.add(i.elementos,beneficio[i.id],peso)
        d2[k].append(item)
        
    
        
resumen = []

for k in d2:
    lista = d2[k]
    for numero in range(len(lista)):
        item = lista[numero]
        if item.peso != 0 and item.peso<(mochila.capacidad-mochila.ocupado):
            p = []
            p.append(k)
            p.append(item.id)
            p.append(item.beneficio)
            p.append(item.peso)
            p.append(item.beneficio/item.peso)
            p.append(numero)
            resumen.append(p)
resumen = np.array(resumen)

pResumen = pd.DataFrame(resumen,columns = ["cluster","id vector","beneficio","costo","beneficio/costo","nid"])
#Se ordena por el cluster
pResumen = pResumen.sort_values(by=['cluster'])


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pResumen.to_string(index=False))

 cluster  id vector  beneficio    costo  beneficio/costo     nid
     0.0        0.0      399.0   6129.0         0.065100     7.0
     0.0        0.0      480.0   7962.0         0.060286    34.0
     0.0        0.0      774.0   7164.0         0.108040    35.0
     0.0        0.0      581.0   8542.0         0.068017    36.0
     0.0        0.0      704.0   7490.0         0.093992    37.0
     0.0        0.0      663.0   7372.0         0.089935    38.0
     0.0        0.0      470.0   9337.0         0.050337    39.0
     0.0        0.0      593.0   7820.0         0.075831    40.0
     0.0        0.0      694.0   8633.0         0.080389    41.0
     0.0        0.0      952.0   9400.0         0.101277    42.0
     0.0        0.0      841.0  10135.0         0.082980    43.0
     0.0        0.0      942.0  10083.0         0.093425    44.0
     0.0        0.0     1065.0   9001.0         0.118320    45.0
     0.0        0.0      872.0  10807.0         0.080688    46.0
     0.0        0.0      

# Datos extras de lo de arriba

In [14]:
columna = pResumen['beneficio']
print(f"El mayor beneficio fue de  {columna.max()}")
columna = pResumen['costo']
print(f"El menor costo es de {columna.min()}")

columna = pResumen['beneficio/costo']
print(f"La mejor relacion beneficio/costo es de {columna.max()}")

#Se obtiene el mejor item
mejor = pResumen.loc[pResumen['beneficio/costo'].idxmax(),['cluster','nid']]
key = int(mejor[0])
nid = int(mejor[1])
lista = d2[key]
item = lista[nid]

print(f"\nLa descripcion del item es la siguiente")

print(f"cluster: {item.id}")
print(f"beneficio: {item.beneficio}")
print(f"costo: {item.peso}")
print(f"beneficio/costo: {item.beneficio/item.peso}")
print(f"elementos: {item.elementos}")



El mayor beneficio fue de  2345.0
El menor costo es de 1006.0
La mejor relacion beneficio/costo es de 0.2730507926237464

La descripcion del item es la siguiente
cluster: 18
beneficio: 844
costo: 3091.0
beneficio/costo: 0.2730507926237464
elementos: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
